# Unsorted Search on Classical Computers

Lets start with how you can search a value in an unsorted list of numbers using classical computers. Let’s take a list of 10 numbers in no particular order, such as, `[3,6,1,9,7,2,5,0,8,4]`. Lets say, you want to search for 8 in this list. Since the list is in no particular order, you will have to check the list items one-by-one till you find a match. If you went from left to right, it would take you 9 tries in this list of 10 numbers.

Checking for the match can be seen as a call to an oracle function, which returns True if the input is the number we are looking for, else returns False. So, the oracle function would be called 9 times before you find 8 in the list. However, if you were searching for 3, then you would be lucky and find it on the first try.

On average, you will need to check a list of $N$ numbers $N/2$ times. In other words, it has time complexity of $O(N)$.

What is exciting though is that on a quantum computer you can perform this search in time-complexity of $O(\sqrt{N})$ using Grover’s algorithm. 

In [ ]:
def oracle_classical(n):
    to_match = 8    # this is the number being searched
    if n == to_match:
        return True
    else:
        return False

In [ ]:
# This is the unsorted list in which a number is searched
numlist = [3,6,1,9,7,8,5,0,2,4]

num_tries = None # indicates if the list does not contain the number being searched

for i,n in enumerate(numlist):
    if oracle_classical(n):
        num_tries = i + 1
print(num_tries)

# Unsorted Search on Quantum Computers: Grover's Algorithm
This notebook implements Grover’s algorithm on  Amazon Braket to achieve the mentioned speedup.  
Grover's algorithm takes advantage of superposition, entanglement, and interference, in quantum computers to get that speedup.

***
## First some code for using *Amazon Braket*

### Module imports for *Amazon Braket*
* the class Circuit is core to building quantum circuits on *Amazon Braket*
* subroutine decorator for defining custom operators (gates)
* LocalSimulator is a quantum computer simulator run on your local compute instance

In [ ]:
# Import Braket libraries
from braket.circuits import Circuit
from braket.circuits.circuit import subroutine
from braket.devices import LocalSimulator
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import numpy as np

### Utility function to run quantum circuit on *Amazon Braket*
This function runs a quantum circuit on a specified device, return the probabilities of measuring qubits as specified in the circuit.  
Note the parameter shots=1000. This is to run the circuit that many times to compute the probabilities of the measured results from the circuit.

In [ ]:
# Function to run quantum task, check the status thereof, and collect results
def get_result(circ,device):

    # submit task: define task (asynchronous)
    task = device.run(circ, shots=1000)

    # Wait for job to complete
    status = task.state()
    print('Status:', status)
    # Only notify the user when there's a status change
    while status != 'COMPLETED':
        status = task.state()
        print('Status:', status)
    print()

    # get result
    result = task.result()

    # get output probabilities
    probs_values = result.values[0]
    
    return probs_values

***

## Solution to Dinner Party problem using Grover's algorithm
Dinner Party problem is about selecting a set of friends to invite to a dinner party, while satisfying certain constraints regarding who can be invited along with who. This is basically a boolean satisfiability problem, which belongs to the NP-complete class of problems.

### Problem instance specification
The dinner party constraints for this instance of the problem are the following
- have to invite fromamong  three friends, creatnamed as 0, 1, and 2
- 0 and 1 get along well together; 0 and 2 get along welll together; but 1 and 2 dont get along well
- to invite friends who will get along well together in the dinner party

### Utility functions for building oracle circuit
These functions help in writing the oracle operator.  
Specifically these functions perform the following operations -
* CTL(opMatrix) - this function creates an operator matrix that adds a control qubit for a given operator opMatrix. The control qubit is the most significant qubit in the resukting operator.
* cnx(targets) - this function uses the CTL() function to create a controlled NOT operator with multiple control qubits. The operator takes the least significant qubit in *targets* as the target of the NOT operation, and rest as control qubits.
* qor(ckt, inplist, out) - this function appends an OR boolean operation to the *ckt* circuit, taking *inplist* as the inputs for the OR, and *out* as the output of that OR.
* qand(ckt,inplist,out) - just as the qor operation above, this appends a boolean AND operation to the circuit *ckt*.
* qnot(ckt,inplist) - this appends NOT boolean operation, i.e., simply X gates, for the target qubits, *inplist*

In [ ]:
## Utility function to add control bit
def CTL(opMatrix):
    (r,c) = opMatrix.shape
    oparr = np.array(opMatrix)
    coparr = np.eye(r*2,dtype=complex)
    for i in range(r,r*2):
        for j in range(r,r*2):
            coparr[i][j] = oparr[i-r][j-r]
    return np.array(coparr,dtype=complex)

# Utility function to create n-controls X gate
@subroutine(register=True)
def cnx(targets):
    # define X gate
    x_gate = np.array([[0.0, 1.0], [1.0, 0.0]], dtype=complex)
    
    cnx_gate = x_gate
    for i in range(len(targets)-1):
        cnx_gate = CTL(cnx_gate)
    # instantiate circuit object
    circ = Circuit()
    # add cnx gate
    circ.unitary(matrix=cnx_gate, targets=targets)
    return circ

# Utility functions to implement boolean gates
def qor(ckt,inplist,out):
    ckt.x(inplist)
    ckt.cnx((inplist+[out]))
    ckt.x(inplist)
    ckt.x(out)

def qand(ckt,inplist,out):
    ckt.cnx((inplist+[out]))
    
def qnot(ckt,inplist):
    ckt.x(inplist)

### Assign qubits for inputs, output, and work registers
As we start putting together the quantum circuit, we first identify the qubits to be used in the circuit, and their role therein.  
Specifically, we group the qubits into three registers as following:
* Input register, *inreg* - these qubits represent the individual friends invittion status, at the end of the algorithm these are readout to get the result
* Output register, *outreg* - this qubit is the output qubit of the oracle operator, this qubit is initialized to |-> state at the initialization to effect the phase kickback
* Work register, *work* - these qubits are a scratch pad area for the oracle operator to implement its logic. These qubits are required to be cleaned up back to 0 state when the oracle operator is done

In [ ]:
inreg = [0,1,2]
work = [3,4,5,6,7]
outreg = [8]

### Grover's circuit components
At the heart of Grover’s algorithm are two operators, the oracle and a reflection operator also known as Grover’s diffusion operator. Additionallly, we need to initialize the input qubits into a uniform superposition, and output qubit into |-> state.
#### Oracle for Dinner Party problem
The party_oracle circuit implements the boolean logic for the constraints specified earlier for this probelm instance. So, per the specified constraints it indicates if the input  combinations of invitees are mutually compatible. Remember it cleans up the work qubits when done.  
The cell below additionally prints out the oracle operator's circuit to help visualize its operation.

In [ ]:
def party_oracle(inreg,outreg,work):
    partyckt = Circuit()
    # Invite ((0 and 1) or (0 and 2)) and NOT (1 and 2)
    qand(partyckt,[inreg[0],inreg[1]], work[0])
    qand(partyckt,[inreg[0],inreg[2]], work[1])
    qor(partyckt,[work[0],work[1]],work[2])
    qand(partyckt,[inreg[1],inreg[2]], work[3])
    partyckt.x(work[3])
    qand(partyckt,[work[2],work[3]], work[4])
    partyckt.cnot(work[4],outreg[0])
    # cleanup the work qubits
    qand(partyckt,[work[2],work[3]], work[4])
    partyckt.x(work[3])
    qand(partyckt,[inreg[1],inreg[2]], work[3])
    qor(partyckt,[work[0],work[1]],work[2])
    qand(partyckt,[inreg[0],inreg[2]], work[1])
    qand(partyckt,[inreg[0],inreg[1]], work[0])
    
    return partyckt
print(party_oracle(inreg, outreg,work))

#### Grover's diffusion operator
The diffusion operator is the trick behind Grover’s operator, it flips every amplitude in the superposition around the mean amplitude. In conjunction with the oracle and phase kick-back, this leads to amplification of the amplitude of the target.  
<<TBD more details - this is the most interesting part!>>

In [ ]:
def diffusion_op(inreg,outreg):
    # diffuser
    diff = Circuit()
    diff.h(inreg)
    diff.x(inreg)
    diff.cnx(inreg+outreg)
    diff.x(outreg)
    diff.x(inreg)
    diff.h(inreg)

    return diff
print(diffusion_op(inreg,outreg))

#### Initializer
Initializer circuit puts input register qubits in uniform superposition of all states by applying H gate to each of those qubits. So, the superposition of all states of the input qubits have equal amplitude at the time of initialization. Also, to effect phase kick-back, the output qubit is put in |-⟩ state by applying X gate followed by H gate.

In [ ]:
def prepare(inreg, outreg):
    prep = Circuit()
    prep.h(inreg)
    prep.x(outreg)
    prep.h(outreg)
    return prep
print(prepare(inreg,outreg))

#### Putting these together to create full quantum circuit for Dinner Party problem
We are now ready to assemble the full quantum circuit using the functions to create the individual components.  
A point to note though, the oracle-diffuser pair requires to be iterated $\pi/4 \sqrt{N/t}$ times, if there are $t$ solutions to the problem. However, for this instance of this problem we cheat a little bit since we know in advance the number of iteration required is 1.  
In a more general implementation you will need to use an additional step using *Quantum counting algorithm* to efficiently count the number of solutions for the given problem. The quantum counting algorithm is based on the quantum phase estimation algorithm. This algorithm is a separate topic in its own right, we will not discuss it in this notebook.

In [ ]:
def grover_ckt(inreg,outreg,work,n_reps):
    gckt = prepare(inreg,outreg)
    for i in range(n_reps):
        ora = party_oracle(inreg,outreg,work)
        gckt.add(ora)

        diff = diffusion_op(inreg,outreg)
        gckt.add(diff)

    gckt.probability(target=inreg)

    return gckt

groverckt = grover_ckt(inreg,outreg,work,n_reps=1)
print(groverckt)

# Running the circuit and displaying results

## Function to display measured probabilities
This is a utility function that displays the outcome of the execution of the quantum circuit in a graphical form.

In [ ]:
def show_probs(measured_qubits,probs):
    num_qubits = len(measured_qubits)

    if 2**num_qubits != len(probs):
        print('mismatch in mnum of qubits measured and number of probabilities')
        return -1

    format_str = '{0:0'+str(num_qubits)+'b}'
    labels = [format_str.format(i) for i in range(len(probs))]
    # plot probabalities
    plt.bar(labels, probs)
    plt.xlabel('bitstrings')
    plt.ylabel('probability')
    plt.xticks(rotation=90)

    return 0

## Running on LocalSimulator
Finally we run the circuit a large number of times to get an estimation of the probabilities of the results.  
Here we use LcocalSimulator as the device to run this circuit.

You should see two possible solutions as $101$ and $110$, i.e., invite friends 0 and 2, or invite friends 1 and 2.

In [ ]:
# set up the local simulator
device = LocalSimulator()

# Run on this device
probs = get_result(groverckt, device)

# Plot the probabilities measured
show_probs(inreg,probs)

# Shut down the notebook
Remember to shutdown the notebook to avoid running up your bill.  
Thank you!